In [0]:
import pandas as pd
import numpy as np
import csv

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from google.colab import files
from IPython.display import clear_output


import time

uploaded = files.upload()

In [0]:
#!export GOOGLE_APPLICATION_CREDENTIALS="content/googlekey.json"
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../content/googlekey.json"

In [0]:

train_news = pd.read_csv('https://raw.githubusercontent.com/jems2019/AlternusVera/master/input_data/train_processed.csv')


def get_sentiment_score(doc):
    client = language.LanguageServiceClient()
    #client = language.LanguageServiceClient.from_service_account_json("content/googlekey.json")

    document = types.Document(content=doc,type=enums.Document.Type.PLAIN_TEXT,language='en')
    sentiment_score = client.analyze_sentiment(document=document).document_sentiment.score
    return sentiment_score

def convert_sentiment_to_polarity(sent):
    if(sent < -.25):
        return -1
    elif(sent > .25):
        return 1
    else:
        return 0

    
get_sentiment_score(train_news.loc[1,'headline_text'])

-0.20000000298023224

In [0]:
#print(train_news.shape)
#print(test_news.shape)
dataTrain = pd.read_csv('https://raw.githubusercontent.com/jems2019/AlternusVera/master/input_data/train_processed.csv')
dataTest = pd.read_csv('https://raw.githubusercontent.com/jems2019/AlternusVera/master/input_data/test_processed.csv')

dataTrain['sentiment'] = 0
dataTest['sentiment'] = 0

dataTrain['polarity'] = 0
dataTest['polarity'] = 0

start = time.time()
for index, row in dataTrain.iterrows():
    sentiment = get_sentiment_score(row['headline_text'])
    if(index%50 == 0):
        clear_output()
        print('done ' + str(index) + '/' + str(dataTrain.shape[0]))
        print('Time elapsed - ' + str(time.time()-start) + ' sec')
    dataTrain.loc[index,'sentiment'] = sentiment
    dataTrain.loc[index, 'polarity'] = convert_sentiment_to_polarity(sentiment)

print('done train')
dataTrain.to_csv('my_train_with_raw.csv')
print(dataTrain.head())
print(dataTest.head())
#my_train = my_data.loc[:,['clean']]

#print(train_news.loc[1,'clean'])
#convert_sentiment_to_polarity(get_sentiment_score(my_data.loc[0,'clean']))

done 10250/10263
Time elapsed - 1829.7728424072266 sec
done train
   Unnamed: 0      jsonid        label  \
0           0   2635.json        false   
1           1  10540.json    half-true   
2           2    324.json  mostly-true   
3           3   1123.json        false   
4           4   9028.json    half-true   

                                       headline_text  \
0  Says the Annies List political group supports ...   
1  When did the decline of coal start? It started...   
2  Hillary Clinton agrees with John McCain "by vo...   
3  Health care reform legislation is likely to ma...   
4  The economic turnaround started at the end of ...   

                              subject         speaker       speakerjobtitle  \
0                            abortion    dwayne-bohac  State representative   
1  energy,history,job-accomplishments  scott-surovell        State delegate   
2                      foreign-policy    barack-obama             President   
3                         he

In [0]:
start = time.time()
for index, row in dataTest.iterrows():
    sentiment = get_sentiment_score(row['headline_text'])
    if(index%50 == 0):
        clear_output()
        print('done - ' + str(index) + '/' + str(dataTest.shape[0]))
        print('Time elapsed - ' + str(time.time()-start) + ' sec')
    dataTest.loc[index,'sentiment'] = sentiment
    dataTest.loc[index, 'polarity'] = convert_sentiment_to_polarity(sentiment)
    
print('done test')
dataTest.to_csv('my_test_with_raw.csv') 


done - 1250/1289
Time elapsed - 211.56073331832886 sec
done test
